# Create AI helper to outsource filling boring html forms

### Audience

This article is intended for programmers who want to learn about the basics of "function calling" capacity of LLMs (Large Language Models) ([OpenAi](https://openai.com/) or [Anthropic](https://www.anthropic.com/)).

### What you will learn?

You will learn how to create a dynamically generated [JSON schema](https://json-schema.org/) function signature that allows LLMs to interact with HTML forms. [JSON schema](https://json-schema.org/) is a powerful tool for defining and validating the structure of a JSON object. We will not use any external libraries, only pure JavaScript code.

### Introduction

I guess we can all agree that filling out forms is a boring and time-consuming task. What if we could create an AI assistant that could fill the form for us, while we can dedicate our time to more constructive tasks like scrolling through the latest TikTok videos or playing the latest games?

The AI assistant will be able to fill the form by calling a function with the form fields as arguments. The function will return a JSON object with the form fields as keys and the values to be filled in the form.

There are an infinite number of forms in the wild. Each form is unique with its own structure and naming conventions. Until recently, it was almost impossible to create a generic assistant that could fill any form. But with the advent of LLMs, we can create one.

### Let's get started

Forms can be very different from each other, but they all are built using common elements like input fields, textareas, checkboxes, radio buttons, etc.

First, we need to identify the form elements and their types. Regardless of the element type, each element is expected to have a "name" attribute that will be later used as a key in the JSON object.

For each element type, we will create a function that will return a fragment of [JSON schema](https://json-schema.org/) defining the element. The [JSON schema](https://json-schema.org/) should contain a description of the element's purpose. This is very useful for LLMs to understand the purpose of the element or expected values. The text for the description will be gathered from the element's label or placeholder attribute.

First we will define some helper functions that will be used to extract element description and format element name.

In [ ]:
const getEleDesc = (ele) => {
  const labelsStr = Array.from(ele.labels)
    .map((l) => l.innerText)
    .join(', ');
  const placeholderStr = ele.placeholder;
  return `${labelsStr} ${placeholderStr}`.trim();
};

function formatName(name) {
  // Property keys should match pattern '^[a-zA-Z0-9_-]{1,64}$'
  return name.replace(/[^a-zA-Z0-9_-]/g, '_').substring(0, 63);
}

function getDescription(ele) {
  const describe_id = ele.getAttribute('aria-describedby');
  if (describe_id) {
    return document.querySelector(`#${describe_id}`).innerText;
  }
}

Here we define functions to create schema for input, select, textarea, checkboxes and radios.

Json schema for each element contains:
- `name`: element's name
- `type`: element type, usually string
- `description`: element description

In case of select element, radio or checkbox elements we also add enum field with all possible values. Particular checkboxes and radio elements should be addressed in special way since they can have multiple values options related to one name.

In [ ]:
function getInputSchema(input) {
  const { name, type, id, value, placeholder, required } = input;
  // Skip input with value
  if (value) return;
  // Skip input without a name
  if (!name) return;

  const schema = {
    name,
    type: 'string',
    description: getEleDesc(select), 
  };
  return [formatName(name), schema, required];
}

function getSelectSchema(select) {
  const { name, required } = select;
  if (!name) return;
  const schema = { 
    name,
    type: 'string', 
    description: getEleDesc(select),
    enum: []
  };
  Array.from(select.options).forEach((option) => {
    // Add the option to the schema
    schema.enum.push(option.value);
  });
  return [formatName(name), schema, required];
}

function getTextareaSchema(textArea) {
  const { name, id, required } = textArea;
  if (!name) return;
  const schema = { 
    name,
    type: 'string',
    description: getEleDesc(textArea)
  };
  return [formatName(name), schema, required];
}

function getCheckboxSchema([name, values]) {
  if (!name.endsWith('[]')) {
    // we have solitary checkbox
    console.log('solitary checkbox', name, values);
  }
  const e = document.querySelector(`[name="${name}"]`);
  const description = getDescription(e);
  const schema = {
    name,
    type: 'array',
    description,
    uniqueItems: true,
    items: {
      oneOf: values,
    },
  };
  return [formatName(name), schema];
}

function getRadioSchema([name, values]) {
  const e = document.querySelector(`[name="${name}"]`);
  const description = getDescription(e);
  const schema = {
    name,
    type: 'string',
    description,
    enum: values.map((v) => v.const),
  };
  return [formatName(name), schema];
}

Now we arrive to the most important function that will generate the schema for each form using all available functions.

```javascript
function generateSchema(form) {
  // Select all form inputs, selects, and textareas
  const inputsEle = form.querySelectorAll(
    `input[type="text"], input[type="email"], input[type="number"], input[type="password"], input[type="tel"], input[type="url"], input[type="date"], input[type="time"], input[type="datetime-local"], input[type="month"], input[type="week"], input[type="color"], input[type="range"], input[type="search"]`
  );
  const checkboxEle = form.querySelectorAll(`input[type="checkbox"]`);
  const radiosEle = form.querySelectorAll(`input[type="radio"]`);
  const selectsEle = form.getElementsByTagName('select');
  const textAreasEle = form.getElementsByTagName('textarea');

  const inputs = Array.from(inputsEle)
    .map((input) => getInputSchema(input))
    .filter((e) => e);

  const checkBoxes = groupByName(Array.from(checkboxEle)).map(getCheckboxSchema);
  const radios = groupByName(Array.from(radiosEle)).map(getRadioSchema);

  const selects = Array.from(selectsEle)
    .map((select) => getSelectSchema(select))
    .filter((e) => e);

  const textAreas = Array.from(textAreasEle)
    .map((textArea) => getTextareaSchema(textArea))
    .filter((e) => e);
  const schemaProps = [...inputs, ...checkBoxes, ...radios, ...selects, ...textAreas];
  const required = schemaProps.filter(([, , r]) => r).map(([name]) => name);

  return {
    name: 'fillup_form',
    description: 'Schema to fill form inputs',
    parameters: {
      type: 'object',
      required: required,
      properties: Object.fromEntries(schemaProps.map(([name, schema]) => [name, schema])),
    },
  };
}
```

This function scans the form for all input elements and creates a schema for each of them. It groups checkboxes and radios by name and creates a schema for each group. It also creates a schema for select and textarea elements. Finally, it creates a JSON schema with all the form elements.

`fillForm` is reasonable to fill the form with the data provided in the JSON object. The function iterates over the JSON object and fill the form fields with corresponding values.

```javascript
function fillForm(formFields, inputData) {
  inputData.forEach(([n, v]) => {
    try {
      const fieldDef = formFields[n];
      const fieldName = fieldDef.name;
      const fieldElement = document.querySelector(`[name="${fieldName}"]`);

      if (fieldElement.type === 'radio') {
        const radio = document.querySelector(`[name="${fieldName}"]`);
        console.log('radio', radio);
        radio.checked = true;
      } else if (fieldElement) {
        fieldElement.value = v;
      }
    } catch (error) {
      console.log('Error filling form', n);
      console.error(error);
    }
  });
}
```



```javascript
async function callOpenAiAPI({ api_key, model, max_tokens, tools, messages }) {
  try {
    const response = await fetch('https://api.openai.com/v1/chat/completions', {
      method: 'POST',
      headers: {
        Authorization: `Bearer ${api_key}`,
        'content-type': 'application/json',
      },
      body: JSON.stringify({
        model: model ?? 'gpt-4',
        max_tokens: max_tokens ?? 1024,
        tools: tools,
        messages: messages,
      }),
    });
    const data = await response.json();
    return data;
  } catch (error) {
    console.error('Error:', error);
  }
}
```
Here we define a function that will call OpenAI API with the provided parameters. The function will return the response from the API.

Finally we got to the point where we can call the OpenAI API to fill the form with fake data. We will create a button that will trigger the process of filling the form.

```javascript

const forms = Array.from(document.getElementsByTagName('form'));

forms.forEach((form) => {
  const button = document.createElement('a');
  button.onclick = async () => {
    const formSchema = generateSchema(form);
    const llm = await callOpenAiAPI({
      api_key,
      tools: [
        {
          type: 'function',
          function: formSchema,
        },
      ],
      tool_choice: 'auto',
      response_format: { type: 'json_object' },
      messages: [{ role: 'user', content: '`fillup_form` with fake data. No additional comments. only json data' }],
    });

    try {
      const rawData = llm.choices[0].message?.tool_calls[0].function.arguments;
      const inputData = Object.entries(JSON.parse(rawData));
      fillForm(formSchema.parameters.properties, inputData);
    } catch (error) {
      const rawData = llm.choices[0].message.content;
      const inputData = Object.entries(JSON.parse(rawData));
      fillForm(formSchema.parameters.properties, inputData);
    }
  };
  button.innerText = 'Fill Form';
  button.style = 'position: absolute; top: 0; right: 0;';
  form.appendChild(button);
});
```

First we iterate all forms on the page and for each form we create a button that will trigger the process of filling the form. When the button is clicked we generate the schema for the form and call the OpenAI API to fill the form with fake data. The response from the API is then used to fill the form.


### Where we can go with it?

This script can be further improved by adding more advanced features like handling file uploads, handling dynamic forms, etc.

- `browser extension` that will automatically fill the forms on the page based on stored profiles.

- use LLM to generate `generate fake data` in scenarios where we need to preserve out privacy.

- We can extend script with some persistance layer to store filled values so we can use them in future.

- In case of very complex forms with big number of fields brake the form into smaller parts (fieldsets) and fill them separately.

### Conclusion

This script should work with most forms. However if form is dynamic (some form elements are change or activated on user input) or uses some advanced features like file upload, it may not work as expected. In such cases you may need to adjust the script to handle these cases.

I hope you found this article useful and that you learned something new. If you have any questions or suggestions, feel free to leave a comment below.
